#### Решение

#### Задание 1

In [1]:
import gspread
from gspread_dataframe import get_as_dataframe
from google.oauth2 import service_account
import pandas as pd
import clickhouse_connect

# Загрузка ключей сервисного аккаунта
SERVICE_ACCOUNT_FILE = 'credentials.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

def get_credentials():
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    return credentials

def get_data(spreadsheet_id, worksheet_name):
    try:
        gc = gspread.authorize(get_credentials())
        spreadsheet = gc.open_by_key(spreadsheet_id)
        worksheet = spreadsheet.worksheet(worksheet_name)
        # Чтение данных только из столбцов с A по N
        data = get_as_dataframe(worksheet, usecols=range(0, 14), evaluate_formulas=True)
        data = data.dropna(subset=data.columns[1])
        return data
    except Exception as error:
        print(f'An error occurred: {error}')
        return None
    
def ingest_brightika():
    spreadsheet_id = '1DtaCO6QkT2ZA8QXbFQGZgf_cy7MGGFykTNn64S1_lU8'
    worksheet_name = 'testovoe'
    table_name = "brightika"

    # Получение данных из Google Таблицы
    data = get_data(spreadsheet_id, worksheet_name)
    df = pd.DataFrame(data.iloc[:, :8])
    df['installed_at'] = pd.to_datetime(df['installed_at'])
    df['created_at'] = pd.to_datetime(df['created_at'])   
    df['reporting_revenue'] = df['reporting_revenue'].astype(float)   
    df['ad_mediation_platform'] = df['ad_mediation_platform'].fillna('')    
        
    # Вставка данных в таблицу
    create_table_sql = """
    CREATE TABLE brightika
    (
        `user_id` String,
        `store` String,
        `app_name_dashboard` String,
        `installed_at` DateTime('UTC'),
        `created_at` DateTime('UTC'),
        `event_name` String,
        `reporting_revenue` Float32,
        `ad_mediation_platform` String  
    )
    ENGINE = MergeTree
    ORDER BY installed_at
    """    

    client = clickhouse_connect.get_client(host='localhost', port='8123', user='default', password= '')
    client.command('DROP TABLE IF EXISTS brightika')
    client.command(create_table_sql)
    client.insert_df('brightika', df)
    print(f"Данные успешно вставлены в таблицу {table_name}.")   

ingest_brightika()

Данные успешно вставлены в таблицу brightika.


#### Задание 2

In [2]:
import clickhouse_connect
import pandas as pd

# Создание клиента для подключения к ClickHouse
client = clickhouse_connect.get_client(host='localhost', port='8123', user='default', password='')

# Выполнение SQL-запроса
query = """
SELECT
    app_name_dashboard,
    store,
    cast(installed_at as date) as installed_date,
    SUM(IF(event_name = 'iap_revenue', IF(dateDiff('second', installed_at, created_at) < 86400, reporting_revenue, 0), 0)) AS iap_revenue_d0,
    SUM(IF(event_name = 'iap_revenue', IF(dateDiff('second', installed_at, created_at) < 86400 * 2, reporting_revenue, 0), 0)) AS iap_revenue_d1,
    SUM(IF(event_name = 'iap_revenue', IF(dateDiff('second', installed_at, created_at) < 86400 * 3, reporting_revenue, 0), 0)) AS iap_revenue_d3,
    SUM(IF(event_name = 'iap_revenue', IF(dateDiff('second', installed_at, created_at) < 86400 * 7, reporting_revenue, 0), 0)) AS iap_revenue_d7,
    SUM(reporting_revenue) AS iap_revenue_total,    
    SUM(IF(ad_mediation_platform NOT LIKE '%sdk%' and ad_mediation_platform <>'', IF(dateDiff('second', installed_at, created_at) < 86400, reporting_revenue, 0), 0)) AS api_ad_revenue_d0,
    SUM(IF(ad_mediation_platform NOT LIKE '%sdk%' and ad_mediation_platform <>'', IF(dateDiff('second', installed_at, created_at) < 86400 * 2, reporting_revenue, 0), 0)) AS api_ad_revenue_d1,
    SUM(IF(ad_mediation_platform NOT LIKE '%sdk%' and ad_mediation_platform <>'', IF(dateDiff('second', installed_at, created_at) < 86400 * 3, reporting_revenue, 0), 0)) AS api_ad_revenue_d3,
    SUM(IF(ad_mediation_platform NOT LIKE '%sdk%' and ad_mediation_platform <>'', IF(dateDiff('second', installed_at, created_at) < 86400 * 7, reporting_revenue, 0), 0)) AS api_ad_revenue_d7,
    SUM(IF(ad_mediation_platform NOT LIKE '%sdk%' and ad_mediation_platform <>'', reporting_revenue, 0)) AS api_ad_revenue_total,    
    SUM(IF(ad_mediation_platform LIKE '%sdk%', IF(dateDiff('second', installed_at, created_at) < 86400, reporting_revenue, 0), 0)) AS sdk_ad_revenue_d0,
    SUM(IF(ad_mediation_platform LIKE '%sdk%', IF(dateDiff('second', installed_at, created_at) < 86400 * 2, reporting_revenue, 0), 0)) AS sdk_ad_revenue_d1,
    SUM(IF(ad_mediation_platform LIKE '%sdk%', IF(dateDiff('second', installed_at, created_at) < 86400 * 3, reporting_revenue, 0), 0)) AS sdk_ad_revenue_d3,
    SUM(IF(ad_mediation_platform LIKE '%sdk%', IF(dateDiff('second', installed_at, created_at) < 86400 * 7, reporting_revenue, 0), 0)) AS sdk_ad_revenue_d7,
    SUM(IF(ad_mediation_platform LIKE '%sdk%', reporting_revenue, 0)) AS sdk_ad_revenue_total
FROM
    brightika
GROUP BY
    app_name_dashboard,
    store,
    cast(installed_at as date)
ORDER BY  
    app_name_dashboard,
    store,
    cast(installed_at as date)
"""
result = client.query_df(query)

# Преобразование результата в DataFrame
df = pd.DataFrame(result)

# Вывод DataFrame
print(df)

    app_name_dashboard   store installed_date  iap_revenue_d0  iap_revenue_d1  \
0     072E512DC68B22B1  google     2021-08-21            0.00            0.00   
1     072E512DC68B22B1  google     2021-12-10            0.00            0.00   
2     072E512DC68B22B1  google     2021-12-26            0.00            0.00   
3     072E512DC68B22B1  google     2021-12-30            0.00            0.00   
4     072E512DC68B22B1  google     2022-01-11            0.00            0.00   
..                 ...     ...            ...             ...             ...   
359   FA0EC8432400452C  google     2022-02-19            0.00            0.00   
360   FA0EC8432400452C  google     2022-02-20            0.00            0.00   
361   FA0EC8432400452C  google     2022-02-21            7.97           13.95   
362   FA0EC8432400452C  google     2022-02-22          163.44          191.36   
363   FA0EC8432400452C  google     2022-02-23           30.90           30.90   

     iap_revenue_d3  iap_re

### Дополнительные задания

1. Как можно было бы оптимизировать следующую схему таблицы:

- PARTITION BY можно сделать по месячно, а не по датам
- PARTITION BY можно сделать по другому столбцу, который часто используется в запросах при фильтрации
- Поменять тип данныз country_code на Int32 
- Проверить на сколько необходимо включать все 9 столбцов в ORDER BY (должны включаться столбцы которые часто используются в WHERE и JOIN)
- Рассмотреть возможность разделение таблицы на несколько таблиц (если например часто требуется анализировать данные на основе агрегированных значений без каких-то артибутов)
- Использовать словарь Clickhouse для замены текстовых значений столбцов на числовые идентификаторы (platform,network,app_name)


2. На основе данных из google sheet напишите SQL запрос для Clickchouse который проранжирует
пользователей по следующим основаниям:

In [3]:
import clickhouse_connect
import pandas as pd

# Создание клиента для подключения к ClickHouse
client = clickhouse_connect.get_client(host='localhost', port='8123', user='default', password='')

# Выполнение SQL-запроса
query = """
    --Найдем максимальное количество дней после установки до события
    with cte_day as (
        SELECT
        user_id,
        created_at,
        dateDiff('day', installed_at, created_at) as max_days_rank
        FROM `default`.brightika
    ),
    cte_max_days as (
        Select
        user_id,
        max_days_rank,
        Row_number() Over (Partition by user_id ORDER BY max_days_rank desc) as rn
        From cte_day
    ),
    --Найдем среднему времени в секундах между любыми событиями
    cte_prev as (
        SELECT
        user_id,
        created_at,
        any(created_at) OVER (PARTITION BY user_id ORDER BY created_at ASC ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING) AS created_at_prev,
        Row_number() Over (Partition by user_id ORDER BY created_at ASC) as rn
        FROM `default`.brightika 
    ),
    cte_diff_between_events_rank as (
        Select
        user_id,
        AVG(dateDiff('second', created_at_prev,created_at)) as diff_between_events_rank
        From cte_prev
        Where rn <> 1
        Group by user_id
    )
    SELECT
    b1.user_id as user_id,
    SUM(reporting_revenue) as total_revenue_rank,
    SUM(IF(event_name = 'ad_revenue', reporting_revenue, 0)) AS ad_revenue_rank,
    SUM(IF(event_name = 'iap_revenue', reporting_revenue, 0)) AS iap_revenue_rank,
    b3.diff_between_events_rank as diff_between_events_rank,
    b2.max_days_rank as max_days_rank
    FROM `default`.brightika as b1
    Inner join cte_max_days as b2 on b2.user_id = b1.user_id and b2.rn = 1
    Inner join cte_diff_between_events_rank as b3 on b3.user_id = b1.user_id
    Group by b1.user_id, b3.diff_between_events_rank, b2.max_days_rank
"""
result = client.query_df(query)

# Преобразование результата в DataFrame
df = pd.DataFrame(result)

# Вывод DataFrame
print(df)

                                      user_id  total_revenue_rank  \
0    a23724d96e41ad93cd383a9214c453c76176a86e           22.467753   
1    277170f84b87741fcf557e205013f3cfcfaf0443            5.980000   
2    fb98f3c99b179a66afa1ade1a6e915f8e31fad8d            0.059877   
3    28a9728ca1a1199eb3fe6860e0ce53dbee84e86c            0.129091   
4    b1be05970e858077540e32d68cf9d6fbe8c5837d            0.028808   
..                                        ...                 ...   
182  bb0a27a7c32a15dc79f9cee9044ac1ff458fb0a1           41.980000   
183  cd4335b3aa133d99f85fba280d7e26eff7005352            0.000000   
184  14ebebb1fe426cc865f0e19fa5cac170e09deadd            0.005738   
185  baded8cd97b53024f704f7f308320e9a3ee4f376            0.020622   
186  b87897f5654eb4425bf7ba216ef4137ae08d4c58            0.011448   

     ad_revenue_rank  iap_revenue_rank  diff_between_events_rank  \
0           0.000000         22.467753                  89575.00   
1           0.000000          5.980